In [1]:
%load_ext autoreload
%autoreload 2
import sys

# instead of creating a package using setup.py or building from a docker/singularity file,
# import the sister directory of src code to be called on in notebook.
# This keeps the notebook free from code to only hold visualizations and is easier to test
# It also helps keep the state of variables clean such that cells aren't run out of order with a mysterious state
sys.path.append("..")

In [2]:
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
import os
import datetime as dt
import xarray as xr
import glob
import metpy.calc as mpcalc
from metpy.units import units

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
def get_flag(hrrr_df):
    stations_ls = hrrr_df["station"].unique()
    one_hour = dt.timedelta(hours=1)
    flag_ls = []

    for station in stations_ls:
        df = hrrr_df[hrrr_df["station"] == station]
        time_ls = df["valid_time"].tolist()
        for now, then in zip(time_ls, time_ls[1:]):
            if now + one_hour == then:
                flag_ls.append(False)
            else:
                flag_ls.append(True)

    flag_ls.append(False)
    hrrr_df["flag"] = flag_ls

    return hrrr_df

In [4]:
def read_hrrr_data():
    """
    Reads and concatenates parquet files containing forecast and error data for HRRR weather models
    for the years 2018 to 2022.

    Returns:
        pandas.DataFrame: of hrrr weather forecast information for each NYSM site.
    """

    years = ["2018", "2019"]
    savedir = "/home/aevans/ai2es/processed_data/HRRR/ny/"

    # create empty lists to hold dataframes for each model
    hrrr_fcast_and_error = []

    # loop over years and read in parquet files for each model
    for year in years:
        for month in np.arange(1, 13):
            str_month = str(month).zfill(2)
            if (
                os.path.exists(
                    f"{savedir}HRRR_{year}_{str_month}_direct_compare_to_nysm_sites_mask_water.parquet"
                )
                == True
            ):
                hrrr_fcast_and_error.append(
                    pd.read_parquet(
                        f"{savedir}HRRR_{year}_{str_month}_direct_compare_to_nysm_sites_mask_water.parquet"
                    )
                )
            else:
                continue

    # concatenate dataframes for each model
    hrrr_fcast_and_error_df = pd.concat(hrrr_fcast_and_error)
    hrrr_fcast_and_error_df = hrrr_fcast_and_error_df.reset_index().dropna()

    # return dataframes for each model
    return hrrr_fcast_and_error_df

In [5]:
def add_tabular(hrrr_df, geo_df, suffix):
    geo_keys = geo_df.keys()

    for i, _ in enumerate(geo_df["station"]):
        for k in geo_keys:
            hrrr_df.loc[
                hrrr_df["station"] == geo_df["station"].iloc[i], f"{k}_{suffix}"
            ] = geo_df[k].iloc[i]

    return hrrr_df

In [6]:
def encode(data, col, max_val):
    data[col + "_sin"] = np.sin(2 * np.pi * data[col] / max_val)
    data[col + "_cos"] = np.cos(2 * np.pi * data[col] / max_val)

    return data

In [7]:
def get_flag():
    stations_ls = hrrr_df["station"].unique()
    one_hour = dt.timedelta(hours=1)
    flag_ls = []

    for station in stations_ls:
        df = hrrr_df[hrrr_df["station"] == station]
        time_ls = df["valid_time"].tolist()
        for now, then in zip(time_ls, time_ls[1:]):
            if now + one_hour == then:
                flag_ls.append(False)
            else:
                flag_ls.append(True)

    flag_ls.append(False)
    hrrr_df["flag"] = flag_ls

    return hrrr_df

In [8]:
def load_nysm_data():
    # these parquet files are created by running "get_resampled_nysm_data.ipynb"
    nysm_path = "/home/aevans/nwp_bias/data/nysm/"

    nysm_1H = []
    for year in np.arange(2018, 2020):
        df = pd.read_parquet(f"{nysm_path}nysm_1H_obs_{year}.parquet")
        df.reset_index(inplace=True)
        nysm_1H.append(df)
    nysm_1H_obs = pd.concat(nysm_1H)
    nysm_1H_obs = nysm_1H_obs.dropna()
    return nysm_1H_obs

In [57]:
df = load_nysm_data()

In [58]:
hrrr_df = read_hrrr_data()

In [59]:
df.reset_index(inplace=True)
df = df.rename(columns={"time_1H": "valid_time"})
mytimes = hrrr_df["valid_time"].tolist()
# nysm_df = df[df['valid_time'].isin(mytimes)]

In [60]:
df.shape

(1110409, 18)

In [61]:
hrrr_df.shape

(1992162, 27)

In [62]:
nysm_cats_path = "/home/aevans/nwp_bias/src/landtype/data/nysm.csv"
nlcd_path = "/home/aevans/nwp_bias/src/correlation/data/nlcd_nam.csv"
aspect_path = "/home/aevans/nwp_bias/src/correlation/data/aspect_nam.csv"
elev_path = "/home/aevans/nwp_bias/src/correlation/data/elev_nam.csv"

In [63]:
nlcd_df = pd.read_csv(nlcd_path)
aspect_df = pd.read_csv(aspect_path)
elev_df = pd.read_csv(elev_path)
nysm_cats_df = pd.read_csv(nysm_cats_path)

In [64]:
nysm_cats_df["climate_division_name"][1]

'Eastern Plateau'

In [65]:
cats = nysm_cats_df["climate_division"].unique()

In [66]:
nysm_cats_df1 = nysm_cats_df[nysm_cats_df["climate_division"] == cats[0]]
category_name = nysm_cats_df1["climate_division_name"].unique()[0]
stations = nysm_cats_df1["stid"].tolist()
hrrr_df1 = hrrr_df[hrrr_df["station"].isin(stations)]
nysm_df1 = df[df["station"].isin(stations)]

master_df = hrrr_df1.merge(nysm_df1, on="valid_time", suffixes=(None, "_nysm"))
# master_df['day_of_year'] = master_df['valid_time'].dt.dayofyear
# master_df = encode(master_df, 'day_of_year', 366)
# master_df = add_tabular(master_df, nlcd_df, 'nlcd')
# master_df = add_tabular(master_df, aspect_df, 'aspect')
# master_df = add_tabular(master_df, elev_df, 'elev')

# master_df.to_parquet(f'/home/aevans/nwp_bias/src/machine_learning/data/rough_parquets/rough_lstm_nysmcat_{category_name}.parquet')

In [70]:
master_df = master_df.drop_duplicates(
    subset=["valid_time", "station", "t2m"], keep="first"
)

In [71]:
master_df

,valid_time,time,station,level_0,index,latitude,longitude,t2m,sh2,d2m,...,td,relh,srad,pres,mslp,wspd_sonic,wmax_sonic,wdir_sonic,precip_total,snow_depth
0,2018-01-01 03:00:00,2018-01-01 01:00:00,ADDI,59218879,326784,42.045955,-77.218867,-19.199194,0.00068,-21.993216,...,-21.994141,76.068916,0.0,964.409973,971.574524,1.247007,1.992487,309.544586,0.0,0.045029
9,2018-01-01 03:00:00,2018-01-01 01:00:00,BELM,58789537,325906,42.236924,-78.054535,-18.324194,0.00057,-23.930716,...,-21.994141,76.068916,0.0,964.409973,971.574524,1.247007,1.992487,309.544586,0.0,0.045029
18,2018-01-01 03:00:00,2018-01-01 01:00:00,COHO,55695145,319578,42.524868,-77.447487,-21.261694,0.00059,-23.680716,...,-21.994141,76.068916,0.0,964.409973,971.574524,1.247007,1.992487,309.544586,0.0,0.045029
27,2018-01-01 03:00:00,2018-01-01 01:00:00,DELE,57912760,324113,42.403769,-78.415993,-19.949194,0.00062,-23.055716,...,-21.994141,76.068916,0.0,964.409973,971.574524,1.247007,1.992487,309.544586,0.0,0.045029
36,2018-01-01 03:00:00,2018-01-01 01:00:00,ELMI,58111294,324519,42.117849,-76.825003,-16.949194,0.00071,-21.180716,...,-21.994141,76.068916,0.0,964.409973,971.574524,1.247007,1.992487,309.544586,0.0,0.045029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638750,2019-12-01 02:00:00,2019-12-01 00:00:00,ELMI,88414728,118863,42.117849,-76.825003,-2.517584,0.00217,-7.639777,...,-7.888123,83.833267,0.0,955.350830,959.093445,1.171268,1.926686,48.666149,0.0,0.003082
638759,2019-12-01 02:00:00,2019-12-01 00:00:00,GROV,86090472,115739,42.494405,-77.942309,-5.142584,0.00203,-9.014777,...,-7.888123,83.833267,0.0,955.350830,959.093445,1.171268,1.926686,48.666149,0.0,0.003082
638768,2019-12-01 02:00:00,2019-12-01 00:00:00,HART,89103672,119789,42.206262,-77.691552,-5.892584,0.00191,-9.952277,...,-7.888123,83.833267,0.0,955.350830,959.093445,1.171268,1.926686,48.666149,0.0,0.003082
638777,2019-12-01 02:00:00,2019-12-01 00:00:00,OLEA,91801416,123415,42.097365,-78.391637,-2.767584,0.00236,-7.014777,...,-7.888123,83.833267,0.0,955.350830,959.093445,1.171268,1.926686,48.666149,0.0,0.003082


In [ ]:
# hrrr_df = read_hrrr_data()
hrrr_df["day_of_year"] = hrrr_df["valid_time"].dt.dayofyear
hrrr_df = encode(hrrr_df, "day_of_year", 366)
hrrr_df = add_tabular(hrrr_df, nlcd_df, "nlcd")
hrrr_df = add_tabular(hrrr_df, aspect_df, "aspect")
hrrr_df = add_tabular(hrrr_df, elev_df, "elev")

In [ ]:
nysm_cats_df["climate_division_name"].unique()[0]

In [ ]:
new_df = pd.read_parquet(
    "/home/aevans/nwp_bias/src/machine_learning/data/rough_lstm_nysmcat_Western Plateau.parquet"
)

In [ ]:
the_keys = new_df.keys()

In [ ]:
for k in the_keys:
    print(k)